# Join Statements in SQL

In [1]:
import sqlite3 as sq3
import pandas as pd

## Create the initial table

In [2]:
create_author_table_query = """
CREATE TABLE authors(
    author_id VARCHAR(36) NOT NULL PRIMARY KEY,
    author_name VARCHAR(12) NOT NULL
)
"""

create_reviews_table_query = """
CREATE TABLE ratings(
    review_id VARCHAR(36) NOT NULL PRIMARY KEY,
    post_id VARCHAR(36) NOT NULL,
    rating INT NOT NULL
)
"""

create_table_query = """
CREATE TABLE blog_post(
    post_id VARCHAR(36) NOT NULL PRIMARY KEY,
    name VARCHAR(22) NOT NULL,
    author_id VARHCHAR(36) NOT NULL,
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
)
"""

### Connect or create then connect the Database

In [10]:
try:
    table_name = '../data/blog_posts'
    conn = sq3.connect(table_name)
except:
    conn = sq3.Connection('blog_posts')
    conn.execute(create_author_table_query)
    conn.execute(create_reviews_table_query)
    conn.execute(create_table_query)
    conn.close()  

## Populate the Table with Data

In [ ]:
authors = [('auth1', 'mark'), ('auth2', 'john')]
blog_posts = [('post1', 'Blog Post 1', authors[0][0]), ('post2', 'Blog Post 2', authors[1][0])]

In [ ]:
def populate_database(authors: list[tuple[int, str]],
                      blog_posts: list[tuple[int, str, int]]):
    """_summary_

    Args:
        authors (list): _description_
        blog_posts (list): _description_
    """
    assert len(authors) == len(blog_posts)
    table_name = '../data/blog_posts'
    conn = sq3.connect(table_name)
    cursor = conn.cursor()
    try:
        cursor = conn.cursor()
        cursor.execute("BEGIN TRANSACTION")
        cursor.executemany("INSERT INTO authors(author_id, author_name) VALUES(?, ?)", authors)
        cursor.executemany("INSERT INTO blog_post(post_id, name, author_id) VALUES(?, ?, ?)", blog_posts)
        conn.commit()
    except sq3.Error as e:
        print(f"Error populating database: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()  
    

In [ ]:
populate_database(authors=authors, blog_posts=blog_posts)

## Selecting Blog Posts and Authors

In [ ]:
select_blog_posts_query = """
SELECT blog_post.post_id, blog_post.name, authors.* FROM blog_post
INNER JOIN authors ON blog_post.author_id = authors.author_id 
"""

In [ ]:
table_name = '../data/blog_posts'
conn = sq3.connect(table_name)

In [ ]:
pandas_generator_object = pd.read_sql(select_blog_posts_query, conn)
pandas_generator_object.head()

,post_id,name,author_id,author_name
0,post1,Blog Post 1,auth1,mark
1,post2,Blog Post 2,auth2,john
